In [1]:
import torch

from torch import nn

from ml_utils import *

## Step 1: We will first create a batch of 2 image-like inputs containing 3 dimensions (channels) with 3 x 3 height and width. The batch shape would be [2, 3, 3, 3]

## Image-like random channels are created using the torch.randint function, which takes a low value and a high value for random sampling and the size attribute.

In [2]:
set_seed( 5842 )

img1_ch1 = torch.randint( 0, 9, size=(3, 3), dtype=torch.float32 )
img1_ch2 = torch.randint( 20, 40, size=(3, 3), dtype=torch.float32 )
img1_ch3 = torch.randint( 80, 99, size=(3, 3), dtype=torch.float32 )

img1 = torch.stack( (img1_ch1, img1_ch2, img1_ch3), dim=0 )


img2_ch1 = torch.randint( 0, 9, size=(3, 3), dtype=torch.float32 )
img2_ch2 = torch.randint( 20, 40, size=(3, 3), dtype=torch.float32 )
img2_ch3 = torch.randint( 80, 99, size=(3, 3), dtype=torch.float32 )

img2 = torch.stack( (img2_ch1, img2_ch2, img2_ch3), dim=0 )

batch = torch.stack( (img1, img2), dim=0 )

print( batch.shape )

batch

Random seed set as 5842
torch.Size([2, 3, 3, 3])


tensor([[[[ 0.,  4.,  8.],
          [ 0.,  5.,  4.],
          [ 8.,  7.,  1.]],

         [[33., 24., 24.],
          [25., 34., 38.],
          [30., 37., 21.]],

         [[96., 91., 84.],
          [87., 98., 91.],
          [96., 85., 82.]]],


        [[[ 5.,  8.,  0.],
          [ 8.,  3.,  5.],
          [ 5.,  3.,  4.]],

         [[24., 36., 29.],
          [25., 24., 28.],
          [22., 33., 29.]],

         [[83., 91., 81.],
          [93., 85., 91.],
          [96., 83., 83.]]]])

## Step 2: We will compute the 2d Batch Normalization manually using the below formula:

<div style="background-color:white">
    <img src="./imgs/normalization_formula.png" />
</div>

## While γ and β are the learnable scale and shift parameters, we will use γ=1 and β=0 values, which are also the default values for the Batch Norm2D. Also, we will use ϵ = 0.00001, which is a default value for Batch Norm2D

## The tensor 'mean' function is used to compute the mean of the values of each 'channel' dimension across the entire batch, as shown below. Likewise, the tensor 'var' function is used to compute the variance of the values of each 'channel' dimension across the entire batch, as shown below.

In [3]:
epsilon=1e-05

'''
The tensor 'mean' function is used to compute the mean of the values of each 'channel' dimension across the entire batch, as shown below.
'''
mean = batch.mean( [0,2,3] )


'''
The tensor 'var' function is used to compute the variance of the values of each 'channel' dimension across the entire batch, as shown below.
'''
var = batch.var( [0,2,3], unbiased=False )


'''
Note that shape of the evaluated mean and var as shown below
'''

print( "Shape of Evaluated Mean: {}\n".format( mean.shape ) )
print( "Evaluated Mean:\n {}".format( mean ) )

print( "Shape of Evaluated Variance: {}\n".format( var.shape ) )
print( "Evaluated Variance:\n {}".format( var ) )

Shape of Evaluated Mean: torch.Size([3])

Evaluated Mean:
 tensor([ 4.3333, 28.6667, 88.6667])
Shape of Evaluated Variance: torch.Size([3])

Evaluated Variance:
 tensor([ 7.4444, 27.5556, 30.0000])


## Step 3: The evaluated mean and variance tensors will have to be reshaped so that they can be used in element-wise computations, as in the normalization formula above.

In [4]:
'''
We will reshape the mean and variance tensors so that they can be used in element-wise computations, as in the normalization formula above.
'''
reshaped_mean =  mean[None, :, None, None]
reshaped_var = var[None, :, None, None] 

print( "Shape of Reshaped Mean: {}\n".format( reshaped_mean.shape ) )
print( "Reshaped Mean:\n\n {}\n".format( reshaped_mean ) )
print( "\n####################################\n" )
print( "Shape of Reshaped Variance: {}\n".format( reshaped_var.shape ) )
print( "Reshaped Variance:\n\n {}".format( reshaped_var ) )

Shape of Reshaped Mean: torch.Size([1, 3, 1, 1])

Reshaped Mean:

 tensor([[[[ 4.3333]],

         [[28.6667]],

         [[88.6667]]]])


####################################

Shape of Reshaped Variance: torch.Size([1, 3, 1, 1])

Reshaped Variance:

 tensor([[[[ 7.4444]],

         [[27.5556]],

         [[30.0000]]]])


## Step 4: We will normalize the batch manually by evaluating the formula.

## Note that broadcasting of the values will be automatically applied when element-wise computations are carried out when any corresponding dimensions, such as height and width, do not match.

## γ=1 and β=0 values are used in the manually evaluated formula.

In [5]:
'''
Note that broadcasting of the values will be automatically applied when element-wise computations are carried out, when any corresponding dimensions such as height and width do not match.

Note that γ=1 and β=0 values are used in the below formula.
'''

manually_normalized = ( batch - reshaped_mean ) / (torch.sqrt( reshaped_var + epsilon ) )

## Manually normalized batch of input tensors must match the tensors normalized by batch normalization 2D module.

In [6]:
'''
Manually normalized batch of input tensors must match the tensors normalized by batch normalization 2D module.
'''

print( "Manually Normalized Batch:\n\n {}".format( manually_normalized ) )

Manually Normalized Batch:

 tensor([[[[-1.5882, -0.1222,  1.3439],
          [-1.5882,  0.2443, -0.1222],
          [ 1.3439,  0.9774, -1.2217]],

         [[ 0.8255, -0.8890, -0.8890],
          [-0.6985,  1.0160,  1.7780],
          [ 0.2540,  1.5875, -1.4605]],

         [[ 1.3389,  0.4260, -0.8520],
          [-0.3043,  1.7040,  0.4260],
          [ 1.3389, -0.6694, -1.2172]]],


        [[[ 0.2443,  1.3439, -1.5882],
          [ 1.3439, -0.4887,  0.2443],
          [ 0.2443, -0.4887, -0.1222]],

         [[-0.8890,  1.3970,  0.0635],
          [-0.6985, -0.8890, -0.1270],
          [-1.2700,  0.8255,  0.0635]],

         [[-1.0346,  0.4260, -1.3997],
          [ 0.7912, -0.6694,  0.4260],
          [ 1.3389, -1.0346, -1.0346]]]])


## Step 5: An instance of the PyTorch Batch Norm 2D module will be used to normalize the input batch.

## An instance of the Batch Norm 2D module is created as below by indicating that the input batch has 3 channel dimensions.

In [7]:
'''
An instance of the Batch Norm 2D module is created as below by indicating that the input batch has 3 channel dimensions. In addition, the 'affine' boolean property is set to False 
to indicate that the scale and shift parameters need not be applied for this simple example case. Batch Norm 2D module by default applied the scale and shift parameters which are updated during training.
'''
bnorm2d = nn.BatchNorm2d( 3 ) 

'''
Batches of inputs are then normalized by the module instance as follows.
'''
bn_model_normalized = bnorm2d( batch )

## Step 6: Compare the manually normalized and module normalized batches of inputs to verify that normalized tensor values match.

## As can be seen from the display of the normalized batches, both manually normalized and module-normalized batches of input tensors match in values.

In [8]:
'''
Note that both manually normalized and module-normalized batches of input tensors match in values.  
'''

print( "Module Normalized Batch:\n\n {}".format( bn_model_normalized ) )

Module Normalized Batch:

 tensor([[[[-1.5882, -0.1222,  1.3439],
          [-1.5882,  0.2443, -0.1222],
          [ 1.3439,  0.9774, -1.2217]],

         [[ 0.8255, -0.8890, -0.8890],
          [-0.6985,  1.0160,  1.7780],
          [ 0.2540,  1.5875, -1.4605]],

         [[ 1.3389,  0.4260, -0.8520],
          [-0.3043,  1.7040,  0.4260],
          [ 1.3389, -0.6694, -1.2172]]],


        [[[ 0.2443,  1.3439, -1.5882],
          [ 1.3439, -0.4887,  0.2443],
          [ 0.2443, -0.4887, -0.1222]],

         [[-0.8890,  1.3970,  0.0635],
          [-0.6985, -0.8890, -0.1270],
          [-1.2700,  0.8255,  0.0635]],

         [[-1.0346,  0.4260, -1.3997],
          [ 0.7912, -0.6694,  0.4260],
          [ 1.3389, -1.0346, -1.0346]]]], grad_fn=<NativeBatchNormBackward0>)


In [9]:
if torch.allclose( manually_normalized, bn_model_normalized ):
    print( "SUCCESS: Both manually normalized and module normalized batches match in values" )
else:
    raise Exception("ERROR: Manually normalized and module normalized batches DO NOT match in values" )

SUCCESS: Both manually normalized and module normalized batches match in values


## Step 7: Verify that the mean and standard deviation of each channel in the normalized batch are zero and one, respectively

## ( Mean and Standard Deviation are expected to be very close to 0 and 1).

In [10]:
print( "Normalized Batch Channel 1 Mean: {}\n".format( torch.mean( bn_model_normalized[ :, 0, :, : ] ) ) )
print( "Normalized Batch Channel 1 Std Deviation: {}\n".format( torch.std( bn_model_normalized[ :, 0, :, : ] ) ) )
print( "\n####################################\n" )
print( "Normalized Batch Channel 2 Mean: {}\n".format( torch.mean( bn_model_normalized[ :, 1, :, : ] ) ) )
print( "Normalized Batch Channel 2 Std Deviation: {}\n".format( torch.std( bn_model_normalized[ :, 1, :, : ] ) ) )

Normalized Batch Channel 1 Mean: -2.6490953430879927e-08

Normalized Batch Channel 1 Std Deviation: 1.0289908647537231


####################################

Normalized Batch Channel 2 Mean: 1.8543667579251633e-07

Normalized Batch Channel 2 Std Deviation: 1.0289913415908813



In [11]:
bnorm2d.running_mean

tensor([0.4333, 2.8667, 8.8667])